In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import pandas as pd

In [2]:
df = pd.read_csv('champions.csv')
def get_champ_vec(champ_name):
    arr = []
    row = df.loc[df['Champion'] == champ_name].iloc[0]
    for i in range(1, 16):
        if i == 7:
            if row[i] == 1:
                arr.append(1)
                arr.append(0)
                arr.append(0)
            elif row[i] == 2:
                arr.append(0)
                arr.append(1)
                arr.append(0)
            elif row[i] == 3:
                arr.append(0)
                arr.append(0)
                arr.append(1)
        else:
            arr.append(row[i] / 10)
    return arr

def process_match(match):
        champs = match.split(',')
        vec = []
        for i in range(10):
            champ_vec = get_champ_vec(champs[i])
            for e in champ_vec:
                vec.append(e)
        if champs[10] == 'true':
            vec.append(1)
        else:
            vec.append(0)
        return vec

class MatchDataset(Dataset):
    def __init__(self, file_name, num_games, skip):
        self.inputs = []
        self.labels = []
        with open('data/matches/' + file_name, 'r') as f:
            matches = f.read().split('\n')
            for i in range(skip, skip+num_games):
                try:
                    match_vec = process_match(matches[i])
                    self.inputs.append(torch.tensor(match_vec[:-1], dtype=torch.float32))
                    self.labels.append(torch.tensor([match_vec[-1]], dtype=torch.float32))
                except:
                    pass
                    
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, idx):
        return self.inputs[idx], self.labels[idx]

In [7]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(170, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 16),
            nn.ReLU(),
            nn.Linear(16, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits
model = NeuralNetwork()

In [4]:
learning_rate = 1e-4
batch_size = 64
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

training_data = MatchDataset('diamond_training_data.txt', 150000, 0)
testing_data = MatchDataset('diamond_training_data.txt', 10000, 150000)
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(testing_data, batch_size=batch_size, shuffle=True)

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    avgLoss = 0
    count = 0
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = torch.reshape(model(X), (X.shape[0],))
        loss = loss_fn(pred, torch.reshape(y, (y.shape[0],)))

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            avgLoss += loss.item()
            count += 1
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    return avgLoss / count

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = torch.reshape(model(X), (X.shape[0],))
            test_loss += loss_fn(pred, torch.reshape(y, (y.shape[0],))).item()
            for i in range(pred.shape[0]):
                guess = 0
                if pred[i] > 0.5:
                    guess = 1
                if guess == y[i]:
                    correct += 1

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return test_loss

In [8]:
epochs = 200
train_losses = []
test_losses = []
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loss = train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loss = test_loop(test_dataloader, model, loss_fn)
    train_losses.append(train_loss)
    test_losses.append(test_losses)
print("Done!")

Epoch 1
-------------------------------
loss: 0.694327  [   64/149994]
loss: 0.704204  [ 6464/149994]
loss: 0.694567  [12864/149994]
loss: 0.703273  [19264/149994]
loss: 0.704242  [25664/149994]
loss: 0.691484  [32064/149994]
loss: 0.694522  [38464/149994]
loss: 0.698154  [44864/149994]
loss: 0.692793  [51264/149994]
loss: 0.696294  [57664/149994]
loss: 0.687717  [64064/149994]
loss: 0.696342  [70464/149994]
loss: 0.706071  [76864/149994]
loss: 0.689858  [83264/149994]
loss: 0.696292  [89664/149994]
loss: 0.692684  [96064/149994]
loss: 0.709491  [102464/149994]
loss: 0.688191  [108864/149994]
loss: 0.691322  [115264/149994]
loss: 0.686390  [121664/149994]
loss: 0.696200  [128064/149994]
loss: 0.683331  [134464/149994]
loss: 0.706800  [140864/149994]
loss: 0.714743  [147264/149994]
Test Error: 
 Accuracy: 49.8%, Avg loss: 0.694642 

Epoch 2
-------------------------------
loss: 0.714360  [   64/149994]
loss: 0.682828  [ 6464/149994]
loss: 0.704445  [12864/149994]
loss: 0.692934  [19264/

loss: 0.706471  [115264/149994]
loss: 0.689496  [121664/149994]
loss: 0.704524  [128064/149994]
loss: 0.694438  [134464/149994]
loss: 0.681173  [140864/149994]
loss: 0.696023  [147264/149994]
Test Error: 
 Accuracy: 49.8%, Avg loss: 0.694720 

Epoch 11
-------------------------------
loss: 0.696158  [   64/149994]
loss: 0.686172  [ 6464/149994]
loss: 0.687873  [12864/149994]
loss: 0.691360  [19264/149994]
loss: 0.697631  [25664/149994]
loss: 0.698085  [32064/149994]
loss: 0.696103  [38464/149994]
loss: 0.696368  [44864/149994]
loss: 0.688280  [51264/149994]
loss: 0.706206  [57664/149994]
loss: 0.698028  [64064/149994]
loss: 0.688100  [70464/149994]
loss: 0.707710  [76864/149994]
loss: 0.694534  [83264/149994]
loss: 0.679863  [89664/149994]
loss: 0.694614  [96064/149994]
loss: 0.701268  [102464/149994]
loss: 0.698076  [108864/149994]
loss: 0.708144  [115264/149994]
loss: 0.699807  [121664/149994]
loss: 0.691167  [128064/149994]
loss: 0.699253  [134464/149994]
loss: 0.699771  [140864/149

loss: 0.694767  [76864/149994]
loss: 0.696304  [83264/149994]
loss: 0.697814  [89664/149994]
loss: 0.689643  [96064/149994]
loss: 0.684534  [102464/149994]
loss: 0.697800  [108864/149994]
loss: 0.698334  [115264/149994]
loss: 0.696338  [121664/149994]
loss: 0.676102  [128064/149994]
loss: 0.696196  [134464/149994]
loss: 0.699648  [140864/149994]
loss: 0.699414  [147264/149994]
Test Error: 
 Accuracy: 49.8%, Avg loss: 0.694718 

Epoch 21
-------------------------------
loss: 0.696159  [   64/149994]
loss: 0.701312  [ 6464/149994]
loss: 0.699718  [12864/149994]
loss: 0.701218  [19264/149994]
loss: 0.689705  [25664/149994]
loss: 0.692931  [32064/149994]
loss: 0.689358  [38464/149994]
loss: 0.697736  [44864/149994]
loss: 0.700789  [51264/149994]
loss: 0.691082  [57664/149994]
loss: 0.697954  [64064/149994]
loss: 0.689591  [70464/149994]
loss: 0.693087  [76864/149994]
loss: 0.696118  [83264/149994]
loss: 0.701404  [89664/149994]
loss: 0.693405  [96064/149994]
loss: 0.694672  [102464/149994]

loss: 0.691256  [38464/149994]
loss: 0.695912  [44864/149994]
loss: 0.694344  [51264/149994]
loss: 0.686168  [57664/149994]
loss: 0.694814  [64064/149994]
loss: 0.706251  [70464/149994]
loss: 0.703134  [76864/149994]
loss: 0.697619  [83264/149994]
loss: 0.694357  [89664/149994]
loss: 0.707771  [96064/149994]
loss: 0.696092  [102464/149994]
loss: 0.689614  [108864/149994]
loss: 0.689780  [115264/149994]
loss: 0.691197  [121664/149994]
loss: 0.689408  [128064/149994]
loss: 0.696063  [134464/149994]
loss: 0.684713  [140864/149994]
loss: 0.691349  [147264/149994]
Test Error: 
 Accuracy: 49.8%, Avg loss: 0.694871 

Epoch 31
-------------------------------
loss: 0.693005  [   64/149994]
loss: 0.687718  [ 6464/149994]
loss: 0.688368  [12864/149994]
loss: 0.696074  [19264/149994]
loss: 0.702971  [25664/149994]
loss: 0.683177  [32064/149994]
loss: 0.689734  [38464/149994]
loss: 0.692587  [44864/149994]
loss: 0.689773  [51264/149994]
loss: 0.705940  [57664/149994]
loss: 0.694482  [64064/149994]


loss: 0.706387  [19264/149994]
loss: 0.686195  [25664/149994]
loss: 0.689577  [32064/149994]
loss: 0.697852  [38464/149994]
loss: 0.694699  [44864/149994]
loss: 0.688011  [51264/149994]
loss: 0.677841  [57664/149994]
loss: 0.689518  [64064/149994]
loss: 0.691094  [70464/149994]
loss: 0.691584  [76864/149994]
loss: 0.681422  [83264/149994]
loss: 0.699631  [89664/149994]
loss: 0.696397  [96064/149994]
loss: 0.687844  [102464/149994]
loss: 0.691103  [108864/149994]
loss: 0.701122  [115264/149994]
loss: 0.699399  [121664/149994]
loss: 0.694838  [128064/149994]
loss: 0.706221  [134464/149994]
loss: 0.693035  [140864/149994]
loss: 0.699636  [147264/149994]
Test Error: 
 Accuracy: 49.8%, Avg loss: 0.694678 

Epoch 41
-------------------------------
loss: 0.692884  [   64/149994]
loss: 0.693083  [ 6464/149994]
loss: 0.699347  [12864/149994]
loss: 0.706488  [19264/149994]
loss: 0.699472  [25664/149994]
loss: 0.689682  [32064/149994]
loss: 0.703201  [38464/149994]
loss: 0.691185  [44864/149994]


...

loss: 0.691120  [115264/149994]
loss: 0.689543  [121664/149994]
loss: 0.694165  [128064/149994]
loss: 0.692185  [134464/149994]
loss: 0.697605  [140864/149994]
loss: 0.699630  [147264/149994]
Test Error: 
 Accuracy: 49.8%, Avg loss: 0.694794 

Epoch 158
-------------------------------
loss: 0.689738  [   64/149994]
loss: 0.709416  [ 6464/149994]
loss: 0.697733  [12864/149994]
loss: 0.686200  [19264/149994]
loss: 0.691151  [25664/149994]
loss: 0.687870  [32064/149994]
loss: 0.686510  [38464/149994]
loss: 0.697735  [44864/149994]
loss: 0.697618  [51264/149994]
loss: 0.689832  [57664/149994]
loss: 0.694451  [64064/149994]
loss: 0.695928  [70464/149994]
loss: 0.691343  [76864/149994]
loss: 0.696020  [83264/149994]
loss: 0.676138  [89664/149994]
loss: 0.694720  [96064/149994]
loss: 0.694239  [102464/149994]
loss: 0.699707  [108864/149994]
loss: 0.681336  [115264/149994]
loss: 0.691381  [121664/149994]
loss: 0.692927  [128064/149994]
loss: 0.686106  [134464/149994]
loss: 0.696194  [140864/14

loss: 0.696666  [76864/149994]
loss: 0.687962  [83264/149994]
loss: 0.697650  [89664/149994]
loss: 0.686587  [96064/149994]
loss: 0.704752  [102464/149994]
loss: 0.689815  [108864/149994]
loss: 0.696009  [115264/149994]
loss: 0.697721  [121664/149994]
loss: 0.694710  [128064/149994]
loss: 0.697699  [134464/149994]
loss: 0.694658  [140864/149994]
loss: 0.694510  [147264/149994]
Test Error: 
 Accuracy: 49.8%, Avg loss: 0.694646 

Epoch 168
-------------------------------
loss: 0.701512  [   64/149994]
loss: 0.696207  [ 6464/149994]
loss: 0.697693  [12864/149994]
loss: 0.707790  [19264/149994]
loss: 0.689338  [25664/149994]
loss: 0.700999  [32064/149994]
loss: 0.704479  [38464/149994]
loss: 0.694767  [44864/149994]
loss: 0.696599  [51264/149994]
loss: 0.698203  [57664/149994]
loss: 0.689806  [64064/149994]
loss: 0.689795  [70464/149994]
loss: 0.687925  [76864/149994]
loss: 0.686201  [83264/149994]
loss: 0.697927  [89664/149994]
loss: 0.696383  [96064/149994]
loss: 0.696314  [102464/149994

loss: 0.704528  [38464/149994]
loss: 0.692773  [44864/149994]
loss: 0.691059  [51264/149994]
loss: 0.702853  [57664/149994]
loss: 0.715959  [64064/149994]
loss: 0.709526  [70464/149994]
loss: 0.702970  [76864/149994]
loss: 0.698083  [83264/149994]
loss: 0.699652  [89664/149994]
loss: 0.687888  [96064/149994]
loss: 0.692694  [102464/149994]
loss: 0.689591  [108864/149994]
loss: 0.692958  [115264/149994]
loss: 0.697772  [121664/149994]
loss: 0.701081  [128064/149994]
loss: 0.699794  [134464/149994]
loss: 0.703081  [140864/149994]
loss: 0.707663  [147264/149994]
Test Error: 
 Accuracy: 49.8%, Avg loss: 0.694718 

Epoch 178
-------------------------------
loss: 0.691144  [   64/149994]
loss: 0.711182  [ 6464/149994]
loss: 0.694646  [12864/149994]
loss: 0.694536  [19264/149994]
loss: 0.696223  [25664/149994]
loss: 0.691271  [32064/149994]
loss: 0.687924  [38464/149994]
loss: 0.694319  [44864/149994]
loss: 0.694228  [51264/149994]
loss: 0.702939  [57664/149994]
loss: 0.697852  [64064/149994]

loss: 0.694578  [19264/149994]
loss: 0.681373  [25664/149994]
loss: 0.689623  [32064/149994]
loss: 0.689172  [38464/149994]
loss: 0.696148  [44864/149994]
loss: 0.706320  [51264/149994]
loss: 0.699730  [57664/149994]
loss: 0.686242  [64064/149994]
loss: 0.694526  [70464/149994]
loss: 0.702785  [76864/149994]
loss: 0.699499  [83264/149994]
loss: 0.687754  [89664/149994]
loss: 0.689734  [96064/149994]
loss: 0.694660  [102464/149994]
loss: 0.689538  [108864/149994]
loss: 0.691256  [115264/149994]
loss: 0.694672  [121664/149994]
loss: 0.692798  [128064/149994]
loss: 0.699639  [134464/149994]
loss: 0.704194  [140864/149994]
loss: 0.691202  [147264/149994]
Test Error: 
 Accuracy: 49.8%, Avg loss: 0.694721 

Epoch 188
-------------------------------
loss: 0.697714  [   64/149994]
loss: 0.693171  [ 6464/149994]
loss: 0.692858  [12864/149994]
loss: 0.689622  [19264/149994]
loss: 0.696532  [25664/149994]
loss: 0.693003  [32064/149994]
loss: 0.704711  [38464/149994]
loss: 0.703296  [44864/149994]

Test Error: 
 Accuracy: 49.8%, Avg loss: 0.694756 

Epoch 197
-------------------------------
loss: 0.680864  [   64/149994]
loss: 0.679610  [ 6464/149994]
loss: 0.686106  [12864/149994]
loss: 0.696342  [19264/149994]
loss: 0.699662  [25664/149994]
loss: 0.696338  [32064/149994]
loss: 0.697758  [38464/149994]
loss: 0.699726  [44864/149994]
loss: 0.689641  [51264/149994]
loss: 0.686231  [57664/149994]
loss: 0.696393  [64064/149994]
loss: 0.689881  [70464/149994]
loss: 0.697828  [76864/149994]
loss: 0.699229  [83264/149994]
loss: 0.689411  [89664/149994]
loss: 0.691298  [96064/149994]
loss: 0.702639  [102464/149994]
loss: 0.696305  [108864/149994]
loss: 0.694297  [115264/149994]
loss: 0.691512  [121664/149994]
loss: 0.693515  [128064/149994]
loss: 0.689766  [134464/149994]
loss: 0.692944  [140864/149994]
loss: 0.703174  [147264/149994]
Test Error: 
 Accuracy: 49.8%, Avg loss: 0.694795 

Epoch 198
-------------------------------
loss: 0.708102  [   64/149994]
loss: 0.699481  [ 6464/149994

In [ ]:
train_losses

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(5, 5))
plt.plot(train_losses, label='Training Loss')
plt.plot(test_losses, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.xscale('log')
plt.ylabel('Loss')
plt.legend()
plt.show()